# Obtaining Daily Historical Weather Data from the Global Historical Climatology Network (GHCN) off of NOAA's Website using Python on DSX Local

By: Michael Travis  
2017-07-27

This notebook provides code to retrieve, clean and format the daily historical weather data from the Global Historical Climatology Network (GHCN) off of NOAA's website (ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/).
The Code belows grabs the weather data from US weather stations over the past 82 years that have 2% or less missing data. Code can be changed to change location, time frame and amount of missing data.

The functions df_ghcnd_inventory(), df_ghcnd_stations(), get_ghcnd_all(), df_ghcnd_all() were heavily derived from the functions in https://github.com/jjrennie/GHCNpy

The cleaning and formatting followed http://spatialreasoning.com/wp/20170307_1244_r-reading-filtering-weather-data-from-the-global-historical-climatology-network-ghcn

## 1. Station Metadata  
This section will obtain the metadata for all the stations by getting the ghcnd-stations.txt and ghcnd-inventory.txt files from the website.  


In [1]:
#import necessary libraries
from ftplib import FTP
import numpy as np
import pandas as pd

In [2]:
#From the readme.txt:  
#  
#IV. FORMAT OF "ghcnd-stations.txt"  
#  
#------------------------------  
#Variable   Columns   Type  
#------------------------------  
#ID            1-11   Character  
#LATITUDE     13-20   Real  
#LONGITUDE    22-30   Real  
#ELEVATION    32-37   Real  
#STATE        39-40   Character  
#NAME         42-71   Character  
#GSN FLAG     73-75   Character  
#HCN/CRN FLAG 77-79   Character  
#WMO ID       81-85   Character  
#------------------------------
#
#creates a dataframe for ghcnd-stations.txt from noaa's website
def df_ghcnd_stations():
    print("GRABBING LATEST STATION METADATA FILE")

    ftp = FTP('ftp.ncdc.noaa.gov')
    ftp.login()
    ftp.cwd('pub/data/ghcn/daily')
    ftp.retrbinary('RETR ghcnd-stations.txt', open('ghcnd-stations.txt', 'wb').write)
    ftp.quit()

    # Read in GHCND-D Stations File
    ghcnd_stnfile = 'ghcnd-stations.txt'
    ghcnd_stations = np.genfromtxt(ghcnd_stnfile,delimiter=(11,9,10,7,1,2,1,30,1,3,1,3,1,5),dtype=str)
    
    #creates a dataframe form the data in the text file
    dataframe = pd.DataFrame(ghcnd_stations)
    
    #deletes empty columns
    deleteCols = [4,6,8,10,12]
    for i in deleteCols:
        del dataframe[i]
    
    #gives columns header names
    headers = ["ID", "LAT", "LON", "ELEV", "ST", "NAME","GSN", "HCN", "WMOID"]
    dataframe.columns = headers
    
    return dataframe

In [3]:
#From the readme.txt:  
#    
#VII. FORMAT OF "ghcnd-inventory.txt"  
#  
#------------------------------  
#Variable   Columns   Type  
#------------------------------  
#ID            1-11   Character  
#LATITUDE     13-20   Real  
#LONGITUDE    22-30   Real  
#ELEMENT      32-35   Character  
#FIRSTYEAR    37-40   Integer  
#LASTYEAR     42-45   Integer  
#------------------------------  
#  
#ELEMENT    is the element type.   There are five core elements as well as a number
#           of addition elements.  
#   
#           The five core elements are:  
#  
#           PRCP = Precipitation (tenths of mm)  
#           SNOW = Snowfall (mm)  
#           SNWD = Snow depth (mm)  
#           TMAX = Maximum temperature (tenths of degrees C)  
#           TMIN = Minimum temperature (tenths of degrees C)  
#         
#FIRSTYEAR and LASTYEAR are the first and last years the weather station recorded that element.
#
#creates a dataframe for ghcnd-inventory.txt from noaa's website
def df_ghcnd_inventory():
    print("GRABBING LATEST STATION INVENTORY FILE")

    ftp = FTP('ftp.ncdc.noaa.gov')
    ftp.login()
    ftp.cwd('pub/data/ghcn/daily')
    ftp.retrbinary('RETR ghcnd-inventory.txt', open('ghcnd-inventory.txt', 'wb').write)
    ftp.quit()

    # Read in GHCND-D INVENTORY File
    ghcnd_invfile='ghcnd-inventory.txt'
    ghcnd_inventory= np.genfromtxt(ghcnd_invfile,delimiter=(11,1,8,1,9,1,4,1,4,1,4),dtype=str)
    
    #creates a dataframe form the data in the text file
    dataframe = pd.DataFrame(ghcnd_inventory)
    
    #deletes empty columns
    deleteCols = [1,3,5,7,9]
    for i in deleteCols:
        del dataframe[i]
    
    #gives columns header names
    headers = ["ID", "LAT", "LON", "ELEM" , "FIRST", "LAST"]
    dataframe.columns = headers
    
    return dataframe

In [4]:
df_stns = df_ghcnd_stations()
df_stns.head()

GRABBING LATEST STATION METADATA FILE


,ID,LAT,LON,ELEV,ST,NAME,GSN,HCN,WMOID
0,ACW00011604,17.1167,-61.7833,10.1,,ST JOHNS COOLIDGE FLD,,,
1,ACW00011647,17.1333,-61.7833,19.2,,ST JOHNS,,,
2,AE000041196,25.3330,55.5170,34.0,,SHARJAH INTER. AIRP,GSN,,41196
3,AEM00041194,25.2550,55.3640,10.4,,DUBAI INTL,,,41194
4,AEM00041217,24.4330,54.6510,26.8,,ABU DHABI INTL,,,41217


In [5]:
df_inv = df_ghcnd_inventory()
df_inv.head()

GRABBING LATEST STATION INVENTORY FILE


,ID,LAT,LON,ELEM,FIRST,LAST
0,ACW00011604,17.1167,-61.7833,TMAX,1949,1949
1,ACW00011604,17.1167,-61.7833,TMIN,1949,1949
2,ACW00011604,17.1167,-61.7833,PRCP,1949,1949
3,ACW00011604,17.1167,-61.7833,SNOW,1949,1949
4,ACW00011604,17.1167,-61.7833,SNWD,1949,1949


In [6]:
#remove spaces from the LAT and LON columns and convert to float
cols = ["LAT", "LON"]
for i in cols:
    df_inv[i] = df_inv.apply(lambda row: float(row[i].strip()),axis=1)

#remove spaces from the FIRST and LAST columns and convert to int
cols = ["FIRST", "LAST"]
for i in cols:
    df_inv[i] = df_inv.apply(lambda row: int(row[i].strip()),axis=1)

#remove spaces from the ID and ELEM columns
cols = ["ID", "ELEM"]
for i in cols:
    df_inv[i] = df_inv.apply(lambda row: row[i].strip(),axis=1)

## 2. Subsetting to Stations of Interest  
This section subsets and merges the df_inv and df_stns dataframes to stations in the US that conatin max and min temperature and precipitation data for at least the past 82 years.

In [7]:
#Subset to only stations in the US
us1 = df_inv[df_inv["ID"].str.contains('US')]

#Drop the LAT and LON columns as the inventory and stations dataframes will be merged below 
#and the stations dataframe already contains LAT and LON
del us1["LAT"]
del us1["LON"]

In [8]:
#Subset to stations that have been monitoring max temperature since 1935 or earlier
us2 = us1[(us1.ELEM == 'TMAX') & (us1.FIRST <= 1935) & (us1.LAST >=2017)]
del us2["ELEM"]
headers = ("ID", "TMAXf", "TMAXl")
us2.columns = headers

#Subset to stations that have been monitoring min temperature since 1935 or earlier
us3 = us1[(us1.ELEM == 'TMIN') & (us1.FIRST <= 1935) & (us1.LAST >=2017)]
del us3["ELEM"]
headers = ("ID", "TMINf", "TMINl")
us3.columns = headers

#Subset to stations that have been monitoring precipitation since 1935 or earlier
us4 = us1[(us1.ELEM == 'PRCP') & (us1.FIRST <= 1935) & (us1.LAST >=2017)]
del us4["ELEM"]
headers = ("ID", "PRCPf", "PRCPl")
us4.columns = headers

In [9]:
us_years = us2.merge(us3, how = 'inner', on = 'ID')
us_years = us_years.merge(us4, how = 'inner', on = 'ID')

In [10]:
#The resulting dataframe looks like this
us_years.head()

,ID,TMAXf,TMAXl,TMINf,TMINl,PRCPf,PRCPl
0,USC00010178,1934,2017,1934,2017,1934,2017
1,USC00010252,1912,2017,1912,2017,1912,2017
2,USC00010583,1915,2017,1915,2017,1913,2017
3,USC00012758,1890,2017,1890,2017,1890,2017
4,USC00012813,1917,2017,1917,2017,1917,2017


In [11]:
#Subset to only stations in the US
us_stns = df_stns[df_stns["ID"].str.contains('US')]

In [12]:
us_stns['ST'].unique()

array(['SD', 'CO', 'NE', 'AK', 'AL', 'AR', 'AZ', 'CA', 'TN', 'CT', 'DC',
       'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA',
       'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NH',
       'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'TX',
       'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY', 'PI', 'UM'], dtype=object)

In [13]:
#Manually remove leftover stations that aren't in the US
conus_stns = us_stns[(us_stns.ST != "AK") & (us_stns.ST != "HI") & (us_stns.ST != "PI") & (us_stns.ST != "UM")]
us82 = us_stns.merge(us_years, how = 'right', on = 'ID')

In [14]:
#The dataframe contains all US stations that have been operating for 82 or more years
us82.head()

,ID,LAT,LON,ELEV,ST,NAME,GSN,HCN,WMOID,TMAXf,TMAXl,TMINf,TMINl,PRCPf,PRCPl
0,USC00010178,33.1272,-88.1550,59.4,AL,ALICEVILLE,,,,1934,2017,1934,2017,1934,2017
1,USC00010252,31.3072,-86.5225,76.2,AL,ANDALUSIA 3 W,,,,1912,2017,1912,2017,1912,2017
2,USC00010583,30.8839,-87.7853,82.6,AL,BAY MINETTE,,,,1915,2017,1915,2017,1913,2017
3,USC00012758,31.4450,-86.9533,88.4,AL,EVERGREEN,,,,1890,2017,1890,2017,1890,2017
4,USC00012813,30.5467,-87.8808,7.0,AL,FAIRHOPE 2 NE,,HCN,,1917,2017,1917,2017,1917,2017


## 3. Daily Weather Files
This section obtains all the daily weather files from the website and converts the ones needed to .csv files.

In [15]:
#From the readme.txt
#
#III. FORMAT OF DATA FILES (".dly" FILES)
#
#Each ".dly" file contains data for one station.  The name of the file
#corresponds to a station's identification code.  For example, "USC00026481.dly"
#contains the data for the station with the identification code USC00026481).
#
#Each record in a file contains one month of daily data.  The variables on each
#line include the following:
#
#------------------------------
#Variable   Columns   Type
#------------------------------
#ID            1-11   Character
#YEAR         12-15   Integer
#MONTH        16-17   Integer
#ELEMENT      18-21   Character
#VALUE1       22-26   Integer
#MFLAG1       27-27   Character
#QFLAG1       28-28   Character
#SFLAG1       29-29   Character
#VALUE2       30-34   Integer
#MFLAG2       35-35   Character
#QFLAG2       36-36   Character
#SFLAG2       37-37   Character
#  .           .          .
#  .           .          .
#  .           .          .
#VALUE31    262-266   Integer
#MFLAG31    267-267   Character
#QFLAG31    268-268   Character
#SFLAG31    269-269   Character
#------------------------------
#
#retrieves ghcnd_all.tar.gz from noaa's website
def get_ghcnd_all():
    print("GRABBING ALL STATION DATA")

    ftp = FTP('ftp.ncdc.noaa.gov')
    ftp.login()
    ftp.cwd('pub/data/ghcn/daily')
    ftp.retrbinary('RETR ghcnd_all.tar.gz', open('ghcnd_all.tar.gz', 'wb').write)
    ftp.quit()

In [16]:
#converts all .dly files to .csv from ghcnd_all.tar
def csv_ghcnd_all(ID):
    for i in range(len(ID)):
        
        # Read in all daily files
        delimit = (11,4,2,4) + (5,1,1,1)*31
        df = np.genfromtxt(("ghcnd_all/"+ID[i]+".dly"),delimiter=delimit,dtype=str, missing_values = "-9999")
        
        #convert to Pandas Dataframe
        dataframe = pd.DataFrame(df)
        
        #xx headers denote columns that will be dropped
        headers = ()
        for j in range(1,32):
            headers = headers + ("Val{}".format(j),"xx","xx","xx")
    
        headers = ("ID", "year", "month", "element") + headers
        dataframe.columns = headers
        
        #Drop unneeded columns
        del dataframe["xx"]
    
        dataframe.to_csv(("ghcnd_all/"+ID[i]+".csv"),index=False)

__Uncomment the code in the four cells below and run each only once.__  
The first, third and fourth blocks below will take a very long time to run, i.e. hours. Running the code multiple times will import files that are already imported and convert files to csv that have already been converted.

In [17]:
#get_ghcnd_all()

In [18]:
#Check the size the file will be after unzipped
#!gunzip -c ghcnd_all.tar.gz | wc --bytes

In [19]:
#unzip file
#!tar xzvf ghcnd_all.tar.gz

In [20]:
#csv_ghcnd_all(us82.ID)

## 4. Filtering Staions with Missing Data
This section filters out the stations that are missing greater than 2% of their data.

In [21]:
#Counts how many NA values there are for each weather station for each weather value
us82["natmax"] = 0
us82["natmin"] = 0
us82["naprcp"] = 0
for i in range(len(us82["ID"])):
    df = pd.read_csv("ghcnd_all/"+us82.ID[i]+".csv")
    df = df.replace(-9999,np.NaN)
    us82.loc[i,"natmax"] = sum(df[df.element == 'TMAX'].iloc[:,4:35].isnull().sum(axis=1).tolist())
    us82.loc[i,"natmin"] = sum(df[df.element == 'TMIN'].iloc[:,4:35].isnull().sum(axis=1).tolist())
    us82.loc[i,'naprcp'] = sum(df[df.element == 'PRCP'].iloc[:,4:35].isnull().sum(axis=1).tolist())
    

In [22]:
#drops stations with more than 2% of data missing (365*12*31*0.02 = 610.08)
stn82 = us82[(us82.natmax<=610) & (us82.natmin<=610) & (us82.naprcp<=610)]
stn82.head()

,ID,LAT,LON,ELEV,ST,NAME,GSN,HCN,WMOID,TMAXf,TMAXl,TMINf,TMINl,PRCPf,PRCPl,natmax,natmin,naprcp
33,USC00026242,31.9353,-109.2186,1664.2,AZ,PARADISE,,,,1906,2017,1906,2017,1906,2017,440,472,470
49,USC00029464,33.7478,-112.5983,509.0,AZ,WITTMANN 4SW,,,,1930,2017,1930,2017,1923,2017,394,400,459
237,USC00057460,38.4039,-106.4236,2578.6,CO,SARGENTS,,,,1900,2017,1900,2017,1899,2017,36,36,542
327,USC00102667,42.4261,-112.1253,1482.9,ID,DOWNEY,,,,1895,2017,1895,2017,1895,2017,141,154,268
346,USC00106424,46.2325,-116.2430,990.0,ID,NEZPERCE,,,,1901,2017,1901,2017,1901,2017,520,519,526


stn82 now conatains all US stations and their IDS that have more than 98% of their data over the past 82 years. You can now use those IDs to retrieve their respective .csv file and perform exploratory data analysis and modelling on the data. An example of doing analysis on the data can be found at http://spatialreasoning.com/wp/20170316_1035_r-analyzing-weather-data-from-the-global-historical-climatology-network-ghcn, the analysis is in R but can be easily translated to Python and provides an idea of how to perform the analysis with multiple .csv files.

# 5. Sources  
1. Menne, M.J., I. Durre, R.S. Vose, B.E. Gleason, and T.G. Houston, 2012:  An overview 
of the Global Historical Climatology Network-Daily Database.  Journal of Atmospheric 
and Oceanic Technology, 29, 897-910, doi:10.1175/JTECH-D-11-00103.1.

2. Menne, M.J., I. Durre, B. Korzeniewski, S. McNeal, K. Thomas, X. Yin, S. Anthony, R. Ray, 
R.S. Vose, B.E.Gleason, and T.G. Houston, 2012: Global Historical Climatology Network - 
Daily (GHCN-Daily), Version 3.22

3. http://spatialreasoning.com/wp/20170307_1244_r-reading-filtering-weather-data-from-the-global-historical-climatology-network-ghcn

4. https://github.com/jjrennie/GHCNpy